# Step 1: Install Requirements

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 14378, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 14378 (delta 1), reused 4 (delta 0), pack-reused 14371
Receiving objects: 100% (14378/14378), 13.60 MiB | 15.93 MiB/s, done.
Resolving deltas: 100% (9902/9902), done.
/content/yolov5
     |████████████████████████████████| 182 kB 29.7 MB/s 
     |████████████████████████████████| 62 kB 212 kB/s 
     |████████████████████████████████| 1.6 MB 23.5 MB/s 
     |████████████████████████████████| 42 kB 769 kB/s 
     |████████████████████████████████| 178 kB 50.0 MB/s 
     |████████████████████████████████| 138 kB 73.1 MB/s 
     |████████████████████████████████| 67 kB 6.0 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 145 kB 71.8 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
Setup complete. Using torch 1.12.1+cu113 (Tesla T4)


# Step 2: Download Our Dataset

We create the dataset in Roboflow, exported in the YOLOv5 format.


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="C07MwEl74KJlbJ4mfSjr")
project = rf.workspace("sean-thammakhoune-cqknw").project("urbancoworks_office")
dataset = project.version(1).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to OSS3-1 in yolov5pytorch:: 100%|██████████| 1791/1791 [00:00<00:00, 3362.57it/s]


In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size --img 416
- **batch:** determine batch size --batch 16
- **epochs:** define the number of training epochs --epochs 30
- **data:** define the data --data file_location/data.yaml
- **weights:** specify a path to weights to start transfer learning from --weights yolov5n.pt for nano (YOLOv5 lite)
- **cache:** cache images for faster training

In [ ]:
!python train.py --img 416 --batch 16 --epochs 2 --data OSS3-1/data.yaml --weights yolov5n.pt --cache

train: weights=yolov5n.pt, cfg=, data=OSS3-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-27-g454dae1 Python-3.8.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.

# Download weights and export

We can now download the .pt file and convert it to a .tflite file for CPU deployment

In [ ]:
#export your model's weights for future use
from google.colab import files
PATH_TO_WEIGHTS = './runs/train/exp/weights/best.pt'
!python export.py --w PATH_TO_WEIGHTS --img 416 --batch 1 --include tflite
files.download('./runs/train/exp/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>